In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import re
import pandas as pd

# README

1st Cell - We establish the websites where each piece of information will be scrapped from. Then we start by scrapping the dates of each ECB meeting 

In [2]:
#We have to use selenium library because ECB's website is of the type scroll and load, therefore the code is not all there
# in the simple html version, in this code we open the website and scroll down until we get the whole source code

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Target URL
url = "https://www.ecb.europa.eu/press/govcdec/mopo/html/index.en.html"

# Configure Chrome driver 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the URL
driver.get(url)

driver.maximize_window()

driver.get(url)

last_height = 0

while True:
    driver.execute_script('window.scrollBy(0, 5000)')
    time.sleep(2)

    new_height = driver.execute_script("return document.body.scrollHeight")
    print(str(new_height)+"-"+str(last_height))

    if(new_height == last_height):
        break

    else:
        last_height = new_height

# Get the complete HTML content
html_content = driver.page_source

soup = BeautifulSoup(html_content, "html.parser")


# Quit the browser
driver.quit()

42334-0
41590-42334
42274-41590
43128-42274
43812-43128
48355-43812
48355-48355


In [ ]:
# Regular expression pattern to match the desired links
pattern = re.compile(r'^/press/pr/date/\d{4}/html/.*\.en\.html$')

# Set to keep track of unique links
unique_links = set()
# List to store the first occurrences
first_occurrences = []

# Find all 'a' tags with 'href' attribute
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    # Check if the link matches the pattern
    if pattern.match(href) and href not in unique_links:
        unique_links.add(href)
        first_occurrences.append(href)

# Print the list of first occurrences
print(first_occurrences)

In [ ]:
#Preallocating the list
text_strings = [None]*len(first_occurrences)

#Loop that fills the list with the text from each of the ECB's meeting
for i in range(len(first_occurrences)):
    test = first_occurrences[i]

    link = 'https://www.ecb.europa.eu' + test

    response = requests.get(link)
    html = response.text

    #Creating Beautiful Soup object
    soup = BeautifulSoup(html, 'html.parser')

    page_text = soup.body.main.get_text(separator=' ', strip=True)

    text_strings[i]=page_text

In [ ]:
# Regular expression to match the pattern and capture the date and the rest of the text
pattern = re.compile(r"(PRESS RELEASE Monetary policy decisions \d{1,2} \w+ \d{4})(.*?)(CONTACT.*)", re.DOTALL)

# Dictionary to map month names to month numbers
month_mapping = {
    "January": "01", "February": "02", "March": "03", "April": "04",
    "May": "05", "June": "06", "July": "07", "August": "08",
    "September": "09", "October": "10", "November": "11", "December": "12"
}

def convert_date_format(date_str):
    # Match the date pattern within the extracted date part
    date_match = re.search(r"(\d{1,2}) (\w+) (\d{4})", date_str)
    if date_match:
        day = date_match.group(1).zfill(2)
        month = month_mapping[date_match.group(2)]
        year = date_match.group(3)
        return f"{day}/{month}/{year}"
    return None

# Lists to store formatted dates and remaining texts
dates = []
texts_after_date = []

# Extract dates and remaining texts
for text in text_strings:
    match = pattern.match(text)
    if match:
        date_str = match.group(1)
        remaining_text = match.group(2).strip()
        formatted_date = convert_date_format(date_str)
        dates.append(formatted_date)
        texts_after_date.append(remaining_text)

# Create a DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Text': texts_after_date
})

# Print the DataFrame
print(df)

In [ ]:
#Saving as a csv file
df.to_csv('press_release.csv', encoding='utf-8')

#Press release
https://www.ecb.europa.eu/press/govcdec/mopo/html/index.en.html

#Press conference
https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/html/index.en.html

# things to remember

1 - Introduce a dummy variable for each central bank president

In [ ]:
# From now on open a new section regular expression or text analysis